# 1. 라이브러리 설치

In [1]:
#pip install pmdarima

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
import datetime
from dateutil.relativedelta import relativedelta
import warnings
import copy
import pmdarima as pm
warnings.filterwarnings(action='ignore')

# 2. 사용자 설정

2-1. default 경로 입력

In [2]:
default_dir = 'C:/19Project/05.TRV/01.Pred_Demand/'
data_dir = default_dir + '0. 데이터/input/'

2-2. 파일 경로, 파일명, 설정값 설정

In [3]:
##### 파일 경로 설정 #####
data_dir_output = default_dir + '0. 데이터/output_20211103_1/'

##### 파일명 설정 #####
calendar_file = '달력 데이터.xlsx'

##### 예측 시작일/기간 설정 #####
# 예측 시작일 설정
pred_start = '2021-08-01'
# 예측 기간 설정
pred_day = 7

##### 예측 입력 데이터 기간 범위 설정 (월 단위) #####
# 평일 예측
period_weekday = 1
# 주말 예측
period_weekend = 1
# 주중휴일 예측
period_holiday = 3

##### 예측 통계량 설정 #####
calculation_type = 'max'    # 'mean'

#### 주중 휴일 training data 일자 개수 지정 (해당 개수보다 적을 경우 토,일 데이터 사용) #####
n_holiday = 5

# 3. 데이터 불러오기

3-1. 파일명 설정

In [5]:
data_file_center = '분석테이블_집계_센터차량.csv'
data_file_taxi = '분석테이블_집계_임차택시.csv'
tmp_zone = 'tmp_zone_list.csv' 
tmp_time = 'tmp_time_list.csv'

3-2. 디렉터리 생성

In [6]:
now = datetime.datetime.now().date().strftime('%Y%m%d')

num = 1

arima_dir = default_dir + '4. 배차 예측_' + now

while os.path.exists(arima_dir + '_' + str(num)):
    num += 1
    
arima_dir_num = arima_dir + '_' + str(num) + '/'
arima_pred_dir = arima_dir_num + 'pred/'
arima_train_dir = arima_dir_num + 'train/'

os.makedirs(os.path.join(arima_dir_num))
os.makedirs(os.path.join(arima_pred_dir))
os.makedirs(os.path.join(arima_train_dir))

3-3. 집계 데이터, 달력 데이터, 구역 리스트, 시각 리스트 불러오기

In [7]:
data_center = pd.read_csv(data_dir_output + data_file_center, encoding='cp949', engine = 'python', parse_dates=[0])
data_taxi = pd.read_csv(data_dir_output + data_file_taxi, encoding='cp949', engine = 'python', parse_dates=[0])

calendar_data = pd.read_excel(data_dir + calendar_file, converters = {0: pd.to_datetime, 1: int})

zone_list = pd.read_csv(data_dir_output + tmp_zone, encoding='cp949', engine = 'python')
time_list = pd.read_csv(data_dir_output + tmp_time, encoding='cp949', engine = 'python')

3-4. 달력 데이터 수정

In [8]:
# 달력 데이터 일자 특성 구분 생성 함수
def f_calendar(x):
    if x in [1, 4, 6] : return '평일'
    elif x in [2, 3, 5, 7]: return '휴일' 
    
calendar_data['일자 특성 구분'] = calendar_data['일자 특성'].apply(lambda x: f_calendar(x))
weekday_mapping = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}  
calendar_data['요일 코드'] = calendar_data['일자'].apply(lambda x: x.weekday())
calendar_data['요일'] = calendar_data['요일 코드'].apply(lambda x: weekday_mapping.get(x))
del calendar_data['일자 특성'], calendar_data['요일 코드']   
calendar_data.columns = ['승차 일자', '승차 일자 특성 구분', '승차 요일']   

# 4. 함수 생성

In [9]:
# 분석 테이블 생성 함수
def create_table_arima(addr_data, addr, w, train_start, pred_start, change):
    global zone_list
    global time_list
    # 승하차 구역
    df_od = zone_list[zone_list['승차 구역명'] == addr]
    # 일자
    date_table = calendar_data[(calendar_data['승차 일자'] >= train_start) & (calendar_data['승차 일자'] < pred_start)]    
    if change == 0:
        date_table = date_table[date_table['요일 정보'] == w]
    elif change == 1:
        date_table = date_table[date_table['승차 요일'].isin(['토', '일'])]
    date_table.reset_index(drop = True, inplace = True)
    date_list = date_table['승차 일자']
    for d in date_list:
        df_od['승차 일자'] = d
        if d == date_list[0]:
            df_date = copy.deepcopy(df_od)
        else:
            df_date = df_date.append(df_od)
    df_date.reset_index(drop = True, inplace = True)
    del date_list, df_od
    # 일자 특성
    df_date = df_date.merge(calendar_data, on = '승차 일자', how = 'left')    
    # 승차 시각   
    h_list = time_list['운행 시간']
    for h in h_list:
        df_date['승차 시각'] = h
        if h == h_list[0]:
            df_h = copy.deepcopy(df_date)
        else:
            df_h = df_h.append(df_date)
    df_h.reset_index(drop = True, inplace = True)
    del df_date
    # 승차 분
    m_list = [0, 30]
    for m in m_list:
        df_h['승차 분'] = m
        if m == m_list[0]:
            df_time = copy.deepcopy(df_h)
        else:
            df_time = df_time.append(df_h)
    df_time.reset_index(drop = True, inplace = True)
    del m_list, df_h
    # 집계 결과와 합치기
    table = df_time.merge(addr_data[['승차 일자', '승차 시각', '승차 분', '승차 구역명', '하차 구역명', '운행 수']],
                        on = ['승차 일자', '승차 시각', '승차 분', '승차 구역명', '하차 구역명'], how = 'left')
    table['운행 수'] = table['운행 수'].fillna(0)
    table['요일 정보'] = table['승차 요일'] + ' / ' + table['승차 일자 특성 구분']
    table.sort_values(['승차 구역명', '하차 구역명', '승차 일자', '승차 시각', '승차 분'], inplace = True)
    table.reset_index(drop = True, inplace = True)
    return(table)


# ARIMA 함수
def arima_pred(df, w, model_type, pred_range, car_type, calculation_type): # car_type : 센터차량, 임차택시 (파일 저장에 사용)
    # 평일, 주말 예측
    if model_type != '주중휴일':
        # 요일 정보 선택
        data_w = df[df['요일 정보'] == w]
        data_w.reset_index(drop = True, inplace = True)
        if model_type == '평일':
            train_start = train_start_weekday
        elif model_type == '주말':
            train_start = train_start_weekend
        change = 0
        if data_w['승차 일자'].min().date() > train_start:
            train_start = data_w['승차 일자'].min().date()   
        ## 수정
        train_start = pd.to_datetime(train_start)
        global pred_start
        pred_start = pd.to_datetime(pred_start)
        data_day = data_w[(data_w['승차 일자'] >= train_start) & (data_w['승차 일자'] < pred_start)]
        data_day.reset_index(drop = True, inplace = True)
    #주중휴일 예측
    elif model_type == '주중휴일':
        data_w = df[(df['승차 요일'].isin(['월', '화', '수', '목', '금'])) & (df['승차 일자 특성 구분'] == '휴일')]
        train_start = train_start_holiday
        change = 0
        if data_w['승차 일자'].min().date() > train_start:
            train_start = data_w['승차 일자'].min().date()    
        data_day = data_w[(data_w['승차 일자'] >= train_start) & (data_w['승차 일자'] < pred_start)]
        data_day.reset_index(drop = True, inplace = True)
        # 예측에 사용할 주중휴일 일자 개수 확인 (지정한 개수 미만인 경우 토,일 데이터 사용하여 예측)
        holiday_list = list(data_day['승차 일자'].unique())
        if (len(holiday_list) < n_holiday) | (len(holiday_list) <= len(pred_range)):
            data_w = df[df['승차 요일'].isin(['토', '일'])]
            train_start = train_start_weekend
            change = 1
            if data_w['승차 일자'].min().date() > train_start:
                train_start = data_w['승차 일자'].min().date()
            data_day = data_w[(data_w['승차 일자'] >= train_start) & (data_w['승차 일자'] < pred_start)]
            data_day.reset_index(drop = True, inplace = True)
    # 승차 구역명 선택
    for addr in data_day['승차 구역명'].unique():
        addr_data = data_day[data_day['승차 구역명'] == addr]
        addr_data.reset_index(drop = True, inplace = True)
        # 분석테이블 생성
        addr_table = create_table_arima(addr_data, addr, w, train_start, pred_start, change)
        # 운행 수 합계 집계
        dep_sum = pd.DataFrame(addr_table.groupby(['승차 일자', '승차 요일', '승차 일자 특성 구분', '요일 정보', 
                                                   '승차 시각', '승차 분', '승차 구역명'])['운행 수'].sum())
        dep_sum.reset_index(inplace = True)
        dep_sum.columns = ['승차 일자', '승차 요일', '승차 일자 특성 구분', '요일 정보', '승차 시각', '승차 분', 
                           '승차 구역명', '운행 수'] 
        # 시간별 운행 수 (mean 또는 max 사용)
        if calculation_type == 'mean':
            dep_cal = pd.DataFrame(dep_sum.groupby(['승차 일자', '승차 요일', '승차 일자 특성 구분', '요일 정보', 
                                                    '승차 시각', '승차 구역명'])['운행 수'].mean())
        elif calculation_type == 'max':
            dep_cal = pd.DataFrame(dep_sum.groupby(['승차 일자', '승차 요일', '승차 일자 특성 구분', '요일 정보',
                                                    '승차 시각', '승차 구역명'])['운행 수'].max())        
        dep_cal.reset_index(inplace = True)
        dep_cal.columns = ['승차 일자', '승차 요일', '승차 일자 특성 구분', '요일 정보', '승차 시각', '승차 구역명',
                           '운행 수_' + calculation_type]
        # 예측 기간 설정
        time_range = time_list['운행 시간']
        for t in time_range:
            pred_range['예측 시각'] = t
            if t == time_range[0]:
                pred_df = copy.deepcopy(pred_range)
            else:
                pred_df = pred_df.append(pred_range)                
        pred_df.reset_index(drop = True, inplace = True)
        pred_df['예측 일시'] = [datetime.datetime.combine(pred_df.loc[i, '승차 일자'], 
                                datetime.time(pred_df.loc[i, '예측 시각'], 0, 0)) for i in pred_df.index]
        pred_df.sort_values('예측 일시', inplace = True)
        pred_df.reset_index(drop = True, inplace = True)
        pred_df = pred_df[['예측 일시']]
        # arima table 생성
        arima_table = dep_cal[['승차 일자', '승차 시각', '승차 구역명', '운행 수_' + calculation_type]]
        arima_table['승차 일시'] = [datetime.datetime.combine(arima_table.loc[i, '승차 일자'], 
                                    datetime.time(arima_table.loc[i, '승차 시각'], 0)) for i in arima_table.index] 
        arima_table.sort_values('승차 일시', inplace = True)
        arima_table.reset_index(drop = True, inplace = True)
        arima_table = arima_table[['승차 일시', '운행 수_' + calculation_type]]
        # arima seasonal 선택
        arima_m = len(time_list)
        # arima 실행
        now = datetime.datetime.now()
        print('[' + now.strftime('%Y/%m/%d %H:%M:%S') + ']    ' + addr + '_' + w.replace(' / ','_') + ' 시작')
        modl = pm.auto_arima(arima_table['운행 수_' + calculation_type], start_p = 0, start_q = 0, m = arima_m, max_p = 2, max_q = 2,
                                start_P = 0, start_Q = 0, max_P = 2, max_Q = 2, seasonal = True, trace = True, 
                                D = 1, error_action = 'ignore', suppress_warnings = True, stepwise = True)
        # 예측값 저장
        preds, conf_int = modl.predict(n_periods = len(pred_df), return_conf_int = True)
        pred_table = pd.DataFrame({'date': pred_df['예측 일시'], 'preds' : preds,
                                    'conf_low' : conf_int[:, 0], 'conf_high' : conf_int[:, 1]})
        pred_table['preds'] = pred_table['preds'].apply(lambda x: 0 if x < 0 else x)
        pred_table['conf_low'] = pred_table['conf_low'].apply(lambda x: 0 if x < 0 else x)
        pred_table['conf_high'] = pred_table['conf_high'].apply(lambda x: 0 if x < 0 else x)
        pred_table['승차 구역명'] = addr
        pred_table['승차 요일 정보'] = w
        # 저장
        arima_name = car_type + '_' + addr + '_' + w.replace(' / ','_')
        arima_table.to_csv(arima_train_dir + arima_name + '_train_data.csv', index = False, encoding = 'MS949')
        pred_table.to_csv(arima_pred_dir + arima_name + '_pred.csv', index = False, encoding = 'MS949')
        now = datetime.datetime.now()
        print('[' + now.strftime('%Y/%m/%d %H:%M:%S') + ']    ' + addr + '_' + w.replace(' / ','_') + '_' + ' 종료........\n')
        
        
# 특별교통수단 배차 차량 예측 모델 함수        
def arima_result(df, car_type, calculation_type = 'max'):
    # 집계 테이블에 요일 정보 추가
    df['요일 정보'] = df['승차 요일'] + ' / ' + df['승차 일자 특성 구분']
    weekday_h = ['월 / 휴일', '화 / 휴일', '수 / 휴일', '목 / 휴일', '금 / 휴일']
    df['요일 정보'] = df['요일 정보'].replace(weekday_h, '주중휴일', regex = True)     
    # 예측일자 설정
    global calendar_data
    calendar_data['요일 정보'] = calendar_data['승차 요일'] + ' / ' + calendar_data['승차 일자 특성 구분']   
    calendar_data['요일 정보'] = calendar_data['요일 정보'].replace(weekday_h, '주중휴일', regex = True)
    calendar = calendar_data[calendar_data['승차 일자'] >= pred_start][:pred_day]      
    calendar.sort_values('승차 일자', inplace = True)
    calendar.reset_index(drop = True, inplace = True)     
    for w in calendar['요일 정보'].unique():
        pred_range = calendar[calendar['요일 정보'] == w]
        pred_range.reset_index(drop = True, inplace = True)
        if pred_range['승차 일자 특성 구분'][0] == '평일':
            model_type = '평일'
        elif (pred_range['승차 요일'][0] == '토') | (pred_range['승차 요일'][0] == '일'):
            model_type = '주말'
        else:
            model_type = '주중휴일'
        arima_pred(df, w, model_type, pred_range, car_type, calculation_type)   

# 5. 배차 차량 예측

5-1. 예측 입력 데이터 시작일 설정

In [10]:
train_start_weekday = datetime.datetime.strptime(pred_start, '%Y-%m-%d').date() - relativedelta(months = period_weekday)
train_start_weekend = datetime.datetime.strptime(pred_start, '%Y-%m-%d').date() - relativedelta(months = period_weekend)
train_start_holiday = datetime.datetime.strptime(pred_start, '%Y-%m-%d').date() - relativedelta(months = period_holiday)

5-2. 센터 차량

In [11]:
arima_result(data_center, '센터차량', calculation_type)

[2021/11/03 13:59:25]    효성동_일_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=90.133, Time=0.13 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=64.545, Time=0.25 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.34 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=88.419, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=81.464, Time=0.04 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=65.254, Time=0.87 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=65.274, Time=0.30 sec
 ARIMA(1,0,0)(0,1,1)[24] intercept   : AIC=inf, Time=0.67 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=67.244, Time=3.14 sec
 ARIMA(0,0,0)(1,1,0)[24] intercept   : AIC=67.680, Time=0.15 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=62.229, Time=0.32 sec
 ARIMA(2,0,0)(0,1,0)[24] intercept   : AIC=74.614, Time=0.06 sec
 ARIMA(2,0,0)(2,1,0)[24] intercept   : AIC=62.641, Time=1.12 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=62.704, Time=0.47 sec
 ARIMA(2,0,0)(0,

 ARIMA(2,1,1)(2,1,0)[24]             : AIC=-27.681, Time=3.15 sec
 ARIMA(2,1,1)(2,1,1)[24]             : AIC=inf, Time=3.25 sec
 ARIMA(2,1,1)(1,1,1)[24]             : AIC=inf, Time=1.25 sec
 ARIMA(1,1,1)(2,1,0)[24]             : AIC=-39.201, Time=3.12 sec
 ARIMA(1,1,1)(1,1,0)[24]             : AIC=145.355, Time=0.29 sec
 ARIMA(1,1,1)(2,1,1)[24]             : AIC=inf, Time=4.18 sec
 ARIMA(1,1,1)(1,1,1)[24]             : AIC=inf, Time=0.85 sec
 ARIMA(0,1,1)(2,1,0)[24]             : AIC=-46.288, Time=2.99 sec
 ARIMA(0,1,1)(1,1,0)[24]             : AIC=145.378, Time=0.14 sec
 ARIMA(0,1,1)(2,1,1)[24]             : AIC=inf, Time=3.21 sec
 ARIMA(0,1,1)(1,1,1)[24]             : AIC=inf, Time=0.60 sec
 ARIMA(0,1,0)(2,1,0)[24]             : AIC=inf, Time=3.43 sec
 ARIMA(0,1,2)(2,1,0)[24]             : AIC=-16.773, Time=2.54 sec
 ARIMA(1,1,2)(2,1,0)[24]             : AIC=-36.730, Time=2.88 sec
 ARIMA(0,1,1)(2,1,0)[24] intercept   : AIC=-36.164, Time=3.90 sec

Best model:  ARIMA(0,1,1)(2,1,0)[24] 

 ARIMA(1,1,0)(1,1,0)[24]             : AIC=124.427, Time=0.11 sec
 ARIMA(0,1,1)(0,1,1)[24]             : AIC=108.004, Time=0.16 sec
 ARIMA(0,1,1)(0,1,0)[24]             : AIC=106.004, Time=0.06 sec
 ARIMA(0,1,1)(1,1,0)[24]             : AIC=108.004, Time=0.15 sec
 ARIMA(0,1,1)(1,1,1)[24]             : AIC=110.004, Time=0.21 sec
 ARIMA(1,1,1)(0,1,0)[24]             : AIC=107.960, Time=0.06 sec
 ARIMA(0,1,2)(0,1,0)[24]             : AIC=107.955, Time=0.06 sec
 ARIMA(1,1,0)(0,1,0)[24]             : AIC=122.427, Time=0.02 sec
 ARIMA(1,1,2)(0,1,0)[24]             : AIC=inf, Time=0.24 sec
 ARIMA(0,1,1)(0,1,0)[24] intercept   : AIC=107.788, Time=0.06 sec

Best model:  ARIMA(0,1,1)(0,1,0)[24]          
Total fit time: 1.159 seconds
[2021/11/03 14:02:44]    연수동_일_휴일_ 종료........

[2021/11/03 14:02:44]    마전동_일_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=-100.600, Time=0.04 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=-96.619, Time=0.24 sec
 ARIMA

 ARIMA(2,0,2)(0,1,1)[24]             : AIC=155.621, Time=0.59 sec
 ARIMA(1,0,0)(0,1,1)[24]             : AIC=164.990, Time=0.16 sec
 ARIMA(1,0,2)(0,1,1)[24]             : AIC=156.601, Time=0.28 sec

Best model:  ARIMA(2,0,1)(0,1,1)[24]          
Total fit time: 22.732 seconds
[2021/11/03 14:03:33]    도화동_월_평일_ 종료........

[2021/11/03 14:03:33]    석남동_월_평일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=163.952, Time=0.03 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=148.218, Time=0.20 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.62 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=162.184, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=165.790, Time=0.04 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=145.589, Time=0.61 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=147.589, Time=0.66 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=143.589, Time=0.45 sec
 ARIMA(0,0

 ARIMA(0,0,0)(2,1,0)[24]             : AIC=inf, Time=0.42 sec
 ARIMA(0,0,0)(2,1,2)[24]             : AIC=inf, Time=0.57 sec
 ARIMA(0,0,0)(1,1,0)[24]             : AIC=166.651, Time=0.08 sec
 ARIMA(0,0,0)(1,1,2)[24]             : AIC=inf, Time=1.61 sec
 ARIMA(1,0,0)(2,1,1)[24]             : AIC=155.065, Time=0.57 sec
 ARIMA(0,0,1)(2,1,1)[24]             : AIC=155.065, Time=0.49 sec
 ARIMA(1,0,1)(2,1,1)[24]             : AIC=157.065, Time=0.60 sec

Best model:  ARIMA(0,0,0)(2,1,1)[24]          
Total fit time: 17.823 seconds
[2021/11/03 14:04:47]    선학동_월_평일_ 종료........

[2021/11/03 14:04:47]    송도동_월_평일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=137.373, Time=0.03 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=122.659, Time=0.21 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.52 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=135.707, Time=0.01 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=139.372, Time=0.03 sec
 ARIMA(1,0,0)(2,1,

...

 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=inf, Time=1.24 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.68 sec
 ARIMA(1,0,0)(0,1,1)[24] intercept   : AIC=inf, Time=0.76 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=3.11 sec
 ARIMA(0,0,0)(1,1,0)[24] intercept   : AIC=-55.851, Time=0.21 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=inf, Time=0.82 sec
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.79 sec
 ARIMA(0,0,0)(0,1,1)[24] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(0,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=1.78 sec
 ARIMA(0,0,1)(1,1,0)[24] intercept   : AIC=-53.852, Time=0.30 sec
 ARIMA(1,0,1)(1,1,0)[24] intercept   : AIC=-51.852, Time=0.56 sec
 ARIMA(0,0,0)(1,1,0)[24]             : AIC=-57.737, Time=0.17 sec
 ARIMA(0,0,0)(2,1,0)[24]             : AIC=inf, Time=0.57 sec
 ARIMA(0,0,0)(1,1,1)[24]             : AIC=2474.730, Time=0.13 sec
 ARIMA(0,0,0)(0,1,1)[24]             : AIC=2472.719, Time=0.11 sec
 ARIMA(0,0,0)(2,1,1)[24]             : AIC=2

 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.72 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=238.466, Time=0.01 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=229.381, Time=0.04 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=184.818, Time=0.67 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=2.48 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=1.04 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=190.282, Time=0.44 sec
 ARIMA(2,0,0)(2,1,0)[24] intercept   : AIC=176.126, Time=0.91 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=179.133, Time=0.28 sec
 ARIMA(2,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=3.31 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=1.11 sec
 ARIMA(2,0,1)(2,1,0)[24] intercept   : AIC=176.183, Time=1.84 sec
 ARIMA(1,0,1)(2,1,0)[24] intercept   : AIC=175.346, Time=1.51 sec
 ARIMA(1,0,1)(1,1,0)[24] intercept   : AIC=177.834, Time=0.48 sec
 ARIMA(1,0,1)(2,1,1)[24] intercept   : AIC=inf, Time=3.34 sec
 ARIMA(1,0,1)(1,1,1)[24] intercept

 ARIMA(1,0,0)(2,1,1)[24]             : AIC=inf, Time=2.78 sec
 ARIMA(1,0,0)(1,1,1)[24]             : AIC=inf, Time=0.80 sec
 ARIMA(0,0,0)(2,1,0)[24]             : AIC=127.997, Time=0.38 sec
 ARIMA(2,0,0)(2,1,0)[24]             : AIC=128.384, Time=0.57 sec
 ARIMA(1,0,1)(2,1,0)[24]             : AIC=127.599, Time=1.15 sec
 ARIMA(0,0,1)(2,1,0)[24]             : AIC=126.574, Time=0.47 sec
 ARIMA(2,0,1)(2,1,0)[24]             : AIC=129.189, Time=1.86 sec

Best model:  ARIMA(1,0,0)(2,1,0)[24]          
Total fit time: 21.168 seconds
[2021/11/03 15:25:02]    효성동_토_휴일_ 종료........

[2021/11/03 15:25:02]    간석동_토_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=251.395, Time=0.06 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=234.202, Time=0.19 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=230.335, Time=0.23 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=252.009, Time=0.02 sec
 ARIMA(0,0,1)(0,1,0)[24] intercept   : AIC=247.537, Time=0.04 sec
 ARIMA(0,0

 ARIMA(0,1,2)(2,1,1)[24]             : AIC=inf, Time=2.88 sec
 ARIMA(0,1,2)(1,1,1)[24]             : AIC=inf, Time=0.91 sec
 ARIMA(0,1,2)(2,1,0)[24] intercept   : AIC=174.993, Time=1.05 sec

Best model:  ARIMA(0,1,2)(2,1,0)[24]          
Total fit time: 31.455 seconds
[2021/11/03 15:26:24]    만수동_토_휴일_ 종료........

[2021/11/03 15:26:24]    다남동_토_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=-95.199, Time=0.02 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=-113.369, Time=0.30 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.67 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=-97.199, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=-93.199, Time=0.03 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=inf, Time=2.69 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.68 sec
 ARIMA(1,0,0)(0,1,1)[24] intercept   : AIC=inf, Time=0.82 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=5.57 sec
 ARIMA(0,0,0)(1,1,0)[24] inte

 ARIMA(0,0,0)(2,1,0)[24]             : AIC=38998.495, Time=0.27 sec
 ARIMA(0,0,0)(1,1,1)[24]             : AIC=inf, Time=1.01 sec
 ARIMA(0,0,0)(0,1,1)[24]             : AIC=inf, Time=0.30 sec
 ARIMA(0,0,0)(2,1,1)[24]             : AIC=39000.500, Time=0.39 sec
 ARIMA(1,0,0)(1,1,0)[24]             : AIC=7.548, Time=0.14 sec
 ARIMA(0,0,1)(1,1,0)[24]             : AIC=7.548, Time=0.13 sec
 ARIMA(1,0,1)(1,1,0)[24]             : AIC=9.548, Time=0.15 sec

Best model:  ARIMA(0,0,0)(1,1,0)[24]          
Total fit time: 7.659 seconds
[2021/11/03 15:28:07]    마전동_토_휴일_ 종료........

[2021/11/03 15:28:07]    연수동_토_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=158.150, Time=0.02 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=67.960, Time=0.27 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.69 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=156.150, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=109.491, Time=0.05 sec
 ARIMA(1,0,0)(2,1,

5-3. 임차 택시

In [12]:
arima_result(data_taxi, '임차택시', calculation_type)

[2021/11/03 15:29:21]    마전동_일_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=65.792, Time=0.04 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=49.031, Time=0.24 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=64.193, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=66.986, Time=0.04 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=42.509, Time=0.85 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=44.501, Time=0.90 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.83 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=41.289, Time=0.52 sec
 ARIMA(0,0,0)(1,1,0)[24] intercept   : AIC=47.256, Time=0.46 sec
 ARIMA(0,0,0)(2,1,1)[24] intercept   : AIC=43.289, Time=0.63 sec
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.71 sec
 ARIMA(0,0,1)(2,1,0)[24] intercept   : AIC=42.417, Time=0.83 sec
 ARIMA(1,0,1)(2,1,0)[24] intercept   : AIC=44.344, Time=1.41 sec
 ARIMA(0,0,0)(2,1,0

 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=38.827, Time=0.23 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=36.257, Time=0.29 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=48.127, Time=0.01 sec
 ARIMA(0,0,1)(0,1,0)[24] intercept   : AIC=51.205, Time=0.03 sec
 ARIMA(0,0,1)(1,1,1)[24] intercept   : AIC=inf, Time=0.72 sec
 ARIMA(0,0,1)(0,1,2)[24] intercept   : AIC=inf, Time=1.44 sec
 ARIMA(0,0,1)(1,1,0)[24] intercept   : AIC=39.008, Time=0.25 sec
 ARIMA(0,0,1)(1,1,2)[24] intercept   : AIC=inf, Time=1.95 sec
 ARIMA(0,0,0)(0,1,1)[24] intercept   : AIC=35.218, Time=0.23 sec
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.51 sec
 ARIMA(0,0,0)(0,1,2)[24] intercept   : AIC=inf, Time=0.95 sec
 ARIMA(0,0,0)(1,1,0)[24] intercept   : AIC=37.973, Time=0.15 sec
 ARIMA(0,0,0)(1,1,2)[24] intercept   : AIC=inf, Time=2.14 sec
 ARIMA(1,0,0)(0,1,1)[24] intercept   : AIC=36.083, Time=0.23 sec
 ARIMA(1,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.81 sec
 ARIMA(0,0,0)(0,1,1)[24]             : AIC=33.

 ARIMA(1,0,1)(0,1,0)[24] intercept   : AIC=-20.040, Time=0.16 sec

Best model:  ARIMA(0,0,0)(0,1,0)[24] intercept
Total fit time: 1.710 seconds
[2021/11/03 15:32:29]    부개동_일_휴일_ 종료........

[2021/11/03 15:32:30]    연수동_일_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,1,0)[24]             : AIC=77.267, Time=0.02 sec
 ARIMA(1,1,0)(1,1,0)[24]             : AIC=73.021, Time=0.17 sec
 ARIMA(0,1,1)(0,1,1)[24]             : AIC=50.432, Time=0.19 sec
 ARIMA(0,1,1)(0,1,0)[24]             : AIC=48.432, Time=0.05 sec
 ARIMA(0,1,1)(1,1,0)[24]             : AIC=50.432, Time=0.16 sec
 ARIMA(0,1,1)(1,1,1)[24]             : AIC=52.432, Time=0.23 sec
 ARIMA(1,1,1)(0,1,0)[24]             : AIC=49.321, Time=0.07 sec
 ARIMA(0,1,2)(0,1,0)[24]             : AIC=48.342, Time=0.06 sec
 ARIMA(0,1,2)(1,1,0)[24]             : AIC=50.342, Time=0.42 sec
 ARIMA(0,1,2)(0,1,1)[24]             : AIC=50.342, Time=0.21 sec
 ARIMA(0,1,2)(1,1,1)[24]             : AIC=52.342, Time=0.31 sec
 ARIMA(1,1,2)(

 ARIMA(0,0,0)(2,1,1)[24]             : AIC=232.892, Time=0.36 sec
 ARIMA(0,0,0)(1,1,1)[24]             : AIC=inf, Time=0.38 sec
 ARIMA(1,0,0)(2,1,0)[24]             : AIC=231.888, Time=0.34 sec
 ARIMA(0,0,1)(2,1,0)[24]             : AIC=231.973, Time=0.33 sec
 ARIMA(1,0,1)(2,1,0)[24]             : AIC=233.734, Time=0.67 sec

Best model:  ARIMA(0,0,0)(2,1,0)[24]          
Total fit time: 7.887 seconds
[2021/11/03 15:33:07]    구월동_월_평일_ 종료........

[2021/11/03 15:33:07]    논현동_월_평일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=212.836, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=192.293, Time=0.21 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.46 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=211.161, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=214.769, Time=0.03 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=193.297, Time=0.61 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=193.296, Time=0.27 sec
 ARIMA(1,0,

 ARIMA(1,0,1)(0,1,1)[24] intercept   : AIC=165.598, Time=0.38 sec
 ARIMA(2,0,2)(0,1,1)[24] intercept   : AIC=160.915, Time=0.66 sec
 ARIMA(2,0,2)(0,1,0)[24] intercept   : AIC=163.335, Time=0.19 sec
 ARIMA(2,0,2)(1,1,1)[24] intercept   : AIC=162.864, Time=1.33 sec
 ARIMA(2,0,2)(0,1,2)[24] intercept   : AIC=162.860, Time=1.77 sec
 ARIMA(2,0,2)(1,1,0)[24] intercept   : AIC=161.972, Time=0.72 sec
 ARIMA(2,0,2)(1,1,2)[24] intercept   : AIC=164.834, Time=2.97 sec
 ARIMA(1,0,2)(0,1,1)[24] intercept   : AIC=161.119, Time=0.35 sec
 ARIMA(2,0,2)(0,1,1)[24]             : AIC=162.677, Time=0.79 sec

Best model:  ARIMA(2,0,2)(0,1,1)[24] intercept
Total fit time: 20.524 seconds
[2021/11/03 15:34:02]    연수동_월_평일_ 종료........

[2021/11/03 15:34:02]    주안동_월_평일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=226.744, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=213.271, Time=0.16 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.63 sec
 ARIMA

...

 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=-3.104, Time=0.34 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.83 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=8.268, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=11.586, Time=0.04 sec
 ARIMA(1,0,0)(2,1,0)[24] intercept   : AIC=-8.121, Time=0.89 sec
 ARIMA(1,0,0)(2,1,1)[24] intercept   : AIC=-7.939, Time=2.68 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=1.14 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=-9.695, Time=0.55 sec
 ARIMA(0,0,0)(1,1,0)[24] intercept   : AIC=-5.065, Time=0.20 sec
 ARIMA(0,0,0)(2,1,1)[24] intercept   : AIC=-8.861, Time=2.53 sec
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.82 sec
 ARIMA(0,0,1)(2,1,0)[24] intercept   : AIC=-8.403, Time=1.12 sec
 ARIMA(1,0,1)(2,1,0)[24] intercept   : AIC=-11.089, Time=3.05 sec
 ARIMA(1,0,1)(1,1,0)[24] intercept   : AIC=-10.481, Time=1.08 sec
 ARIMA(1,0,1)(2,1,1)[24] intercept   : AIC=-8.885, Time=4.08 sec
 ARIMA(1,0,1)(1,1,1)[24] intercep

 ARIMA(0,0,1)(1,1,2)[24] intercept   : AIC=inf, Time=1.47 sec
 ARIMA(0,0,0)(0,1,1)[24] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(1,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=1.22 sec
 ARIMA(0,0,2)(0,1,1)[24] intercept   : AIC=57.878, Time=0.41 sec
 ARIMA(1,0,0)(0,1,1)[24] intercept   : AIC=56.609, Time=0.29 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=84.162, Time=0.05 sec
 ARIMA(1,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=1.46 sec
 ARIMA(1,0,0)(0,1,2)[24] intercept   : AIC=inf, Time=1.98 sec
 ARIMA(1,0,0)(1,1,2)[24] intercept   : AIC=inf, Time=1.54 sec
 ARIMA(2,0,0)(0,1,1)[24] intercept   : AIC=57.498, Time=0.45 sec
 ARIMA(2,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=1.16 sec
 ARIMA(1,0,0)(0,1,1)[24]             : AIC=55.286, Time=0.23 sec
 ARIMA(1,0,0)(0,1,0)[24]             : AIC=82.218, Time=0.03 sec
 ARIMA(1,0,0)(1,1,1)[24]             : AIC=inf, Time=0.80 sec
 ARIMA(1,0,0)(0,1,2)[24]             : AIC=inf, Time=1.04 sec
 ARIMA(1,0,0)(1,1,0)[24]             : AIC=64.469, T

 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=-158.749, Time=0.46 sec
 ARIMA(1,0,0)(0,1,0)[24] intercept   : AIC=-160.757, Time=0.05 sec
 ARIMA(0,0,1)(0,1,0)[24] intercept   : AIC=-160.757, Time=0.08 sec
 ARIMA(1,0,1)(0,1,0)[24] intercept   : AIC=-158.757, Time=0.05 sec

Best model:  ARIMA(0,0,0)(0,1,0)[24]          
Total fit time: 2.292 seconds
[2021/11/03 16:41:57]    서운동_토_휴일_ 종료........

[2021/11/03 16:41:57]    문학동_토_휴일 시작
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[24] intercept   : AIC=104.177, Time=0.02 sec
 ARIMA(1,0,0)(1,1,0)[24] intercept   : AIC=102.729, Time=0.24 sec
 ARIMA(0,0,1)(0,1,1)[24] intercept   : AIC=inf, Time=0.59 sec
 ARIMA(0,0,0)(0,1,0)[24]             : AIC=102.427, Time=0.01 sec
 ARIMA(0,0,0)(1,1,0)[24] intercept   : AIC=100.766, Time=0.15 sec
 ARIMA(0,0,0)(2,1,0)[24] intercept   : AIC=87.408, Time=0.59 sec
 ARIMA(0,0,0)(2,1,1)[24] intercept   : AIC=inf, Time=2.10 sec
 ARIMA(0,0,0)(1,1,1)[24] intercept   : AIC=inf, Time=0.84 sec
 ARIMA(1,0,0

 ARIMA(1,1,0)(2,1,0)[24]             : AIC=inf, Time=0.83 sec
 ARIMA(1,1,0)(1,1,1)[24]             : AIC=inf, Time=0.92 sec
 ARIMA(1,1,0)(0,1,1)[24]             : AIC=inf, Time=0.42 sec
 ARIMA(1,1,0)(2,1,1)[24]             : AIC=inf, Time=2.41 sec
 ARIMA(0,1,0)(1,1,0)[24]             : AIC=224.205, Time=0.08 sec
 ARIMA(2,1,0)(1,1,0)[24]             : AIC=209.052, Time=0.14 sec
 ARIMA(1,1,1)(1,1,0)[24]             : AIC=197.925, Time=0.23 sec
 ARIMA(1,1,1)(0,1,0)[24]             : AIC=205.507, Time=0.05 sec
 ARIMA(1,1,1)(2,1,0)[24]             : AIC=150.984, Time=1.10 sec
 ARIMA(1,1,1)(2,1,1)[24]             : AIC=inf, Time=2.87 sec
 ARIMA(1,1,1)(1,1,1)[24]             : AIC=inf, Time=1.16 sec
 ARIMA(0,1,1)(2,1,0)[24]             : AIC=149.280, Time=1.49 sec
 ARIMA(0,1,1)(1,1,0)[24]             : AIC=198.050, Time=0.14 sec
 ARIMA(0,1,1)(2,1,1)[24]             : AIC=inf, Time=2.26 sec
 ARIMA(0,1,1)(1,1,1)[24]             : AIC=inf, Time=1.03 sec
 ARIMA(0,1,0)(2,1,0)[24]             : AIC

 ARIMA(1,0,1)(1,1,0)[24] intercept   : AIC=-95.853, Time=0.87 sec
 ARIMA(1,0,1)(0,1,1)[24] intercept   : AIC=-95.855, Time=0.67 sec
 ARIMA(1,0,1)(1,1,1)[24] intercept   : AIC=-93.855, Time=0.84 sec
 ARIMA(2,0,1)(0,1,0)[24] intercept   : AIC=-116.955, Time=0.15 sec
 ARIMA(2,0,1)(1,1,0)[24] intercept   : AIC=-114.956, Time=0.96 sec
 ARIMA(2,0,1)(0,1,1)[24] intercept   : AIC=-114.956, Time=0.93 sec
 ARIMA(2,0,1)(1,1,1)[24] intercept   : AIC=-112.956, Time=1.39 sec
 ARIMA(2,0,0)(0,1,0)[24] intercept   : AIC=-118.953, Time=0.09 sec
 ARIMA(2,0,0)(1,1,0)[24] intercept   : AIC=-116.954, Time=0.59 sec
 ARIMA(2,0,0)(0,1,1)[24] intercept   : AIC=-116.954, Time=0.71 sec
 ARIMA(2,0,0)(1,1,1)[24] intercept   : AIC=-114.954, Time=0.78 sec
 ARIMA(2,0,0)(0,1,0)[24]             : AIC=-120.255, Time=0.09 sec
 ARIMA(2,0,0)(1,1,0)[24]             : AIC=-118.255, Time=0.25 sec
 ARIMA(2,0,0)(0,1,1)[24]             : AIC=-118.255, Time=0.15 sec
 ARIMA(2,0,0)(1,1,1)[24]             : AIC=-116.255, Time=0.22 se